<a href="https://colab.research.google.com/github/MaguireMaName/Coursera_Capstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone: Battle of the Neighbourhoods

##Introduction/Business Problem

## Data description

In [0]:
 #!pip install geocoder

In [0]:
# load dependancies

import pandas as pd
import numpy as np
import geocoder
import folium
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [89]:
cbr = pd.read_csv('Canberra suburbs.csv')
print(cbr.shape)
cbr.head()

(154, 4)


,Neighborhood,Postcode,Country,Region
0,Barton,2600,Australia,Australian Capital Territory
1,Canberra,2600,Australia,Australian Capital Territory
2,Page,2614,Australia,Australian Capital Territory
3,City,2601,Australia,Australian Capital Territory
4,Canberra,2601,Australia,Australian Capital Territory


In [0]:
Lat_list=[]
Lng_list=[]

for i in range(cbr.shape[0]):
    address='{}, Canberra, Australia'.format(cbr.at[i,'Neighborhood'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [91]:
for i in range(cbr.shape[0]): 
  
    cbr['Latitude'] = Lat_list
    cbr['Longitude'] = Lng_list
    
print(cbr.shape)
cbr.head()

(154, 6)


,Neighborhood,Postcode,Country,Region,Latitude,Longitude
0,Barton,2600,Australia,Australian Capital Territory,-35.30829,149.13354
1,Canberra,2600,Australia,Australian Capital Territory,-35.30654,149.12655
2,Page,2614,Australia,Australian Capital Territory,-35.23954,149.04826
3,City,2601,Australia,Australian Capital Territory,-35.28007,149.13093
4,Canberra,2601,Australia,Australian Capital Territory,-35.30654,149.12655


In [92]:
address = 'Canberra, Australian Capital Territory'

geolocator = Nominatim(user_agent="canberra_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Canberra using latitude and longitude values
map_cbr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(cbr['Latitude'], cbr['Longitude'], cbr['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(map_cbr)  
    
map_cbr

In [0]:
# define Foursquare Credentials and Version

client_id = 'KL5SVGOS40RKZBQK4G1VXYBKBICWCDQL2NMCASHFYER432SS' #  Foursquare ID'
client_secret = '1A5KPYJQIATH0SDZXPPZ5YK0SHLBYVEGPER5AAIIMDXLZ0AB' #  Foursquare Secret
version = '20180604'
limit = 30

In [0]:
# let's create a function to repeat the same process to all the neighborhoods in toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [95]:

# run the above function on each neighborhood and create a new dataframe called toronto_venues

cbr_venues = getNearbyVenues(names=cbr['Neighborhood'],
                                   latitudes=cbr['Latitude'],
                                   longitudes=cbr['Longitude']
                                  )

Barton
Canberra
Page
City
Canberra
Barton
Hmas Creswell
Jervis Bay
Wreck Bay
Hmas Harman
Parliament House
Yarralumla
Harman
Capital Hill
Russell
Parkes
Deakin
Acton
Watson
Hackett
Ainslie
O'Connor
Downer
Dickson
Lyneham
Manuka
Forrest
Griffith
Red Hill
Kingston
Narrabundah
Causeway
Garran
Hughes
Curtin
Swinger Hill
Woden
O'Malley
Chifley
Lyons
Phillip
Torrens
Isaacs
Mawson
Pearce
Farrer
Civic Square
Canberra Airport
Majura
Pialligo
Fyshwick
Canberra Mc
Uriarra
Uriarra Village
Wright
Duffy
Weston Creek
Weston
Coree
Fisher
Coombs
Stromlo
Turner
Braddon
Campbell
Reid
Jamison Centre
Weetangera
Scullin
Macquarie
Cook
Aranda
Hawker
Kippax Centre
Kippax
Florey
Dunlop
Macgregor
Latham
Charnwood
Fraser
Melba
Flynn
Higgins
Holt
Spence
Belconnen
Belconnen DC
Bruce
Lawson
Belconnen
Hall
Paddys River
Kowen
Hume
Oaks Estate
Beard
Kambah Village
Oxley
Macarthur
Monash
Crace
Kinlyside
Franklin
Taylor
Casey
Moncrieff
Harrison
Jacka
Forde
Bonner
Australian National University
Deakin West
Duntroon
Black 

In [96]:
# check dimensions and data

print(cbr_venues.shape)
cbr_venues.head()

(904, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barton,-35.30829,149.13354,Ottoman Cuisine,-35.305615,149.136704,Turkish Restaurant
1,Barton,-35.30829,149.13354,Ostani,-35.311509,149.133533,Hotel Bar
2,Barton,-35.30829,149.13354,Little Bird,-35.306099,149.135331,Café
3,Barton,-35.30829,149.13354,LiloTang,-35.311991,149.133847,Japanese Restaurant
4,Barton,-35.30829,149.13354,National Archives of Australia,-35.304637,149.131004,History Museum


In [97]:
# the number of venues returned for each neighborhood

cbr_venues.groupby('Neighborhood').count()

print('There are {} unique venue categories.'.format(len(cbr_venues['Venue Category'].unique())))
print(cbr_venues.head())

There are 171 unique venue categories.
  Neighborhood  Neighborhood Latitude  ...  Venue Longitude       Venue Category
0       Barton              -35.30829  ...       149.136704   Turkish Restaurant
1       Barton              -35.30829  ...       149.133533            Hotel Bar
2       Barton              -35.30829  ...       149.135331                 Café
3       Barton              -35.30829  ...       149.133847  Japanese Restaurant
4       Barton              -35.30829  ...       149.131004       History Museum

[5 rows x 7 columns]


In [98]:
# analyse each neighbourhood

# one hot encoding
cbr_onehot = pd.get_dummies(cbr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cbr_onehot['Neighborhood'] = cbr_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cbr_onehot.columns[-1]] + list(cbr_onehot.columns[:-1])
cbr_onehot = cbr_onehot[fixed_columns]

cbr_onehot.head()

,Neighborhood,Airport,Airport Lounge,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bakery,Bar,Baseball Field,Beer Bar,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cricket Ground,Cupcake Shop,...,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Soccer Field,Social Club,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Barton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Barton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Barton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Barton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [99]:
cbr_grouped = cbr_onehot.groupby('Neighborhood').mean().reset_index()
cbr_grouped

,Neighborhood,Airport,Airport Lounge,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bakery,Bar,Baseball Field,Beer Bar,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cricket Ground,Cupcake Shop,...,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Soccer Field,Social Club,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Acton,0.000,0.00,0.0,0.0,0.000000,0.00,0.125000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Ainslie,0.000,0.00,0.0,0.0,0.000000,0.00,0.142857,0.000000,0.142857,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.142857,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Amaroo,0.000,0.00,0.0,0.0,0.000000,0.50,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aranda,0.000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.200,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,Australian National University,0.000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.033333,0.000000,0.000000,0.0,0.00,0.033333,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.200000,0.000000,0.033333,0.000000,0.0,0.000000,...,0.033333,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.033333,0.0,0.033333,0.000000,0.0,0.0,

In [100]:
# top 3 frequencies

num_top_venues = 3

for hood in cbr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cbr_grouped[cbr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acton----
                   venue  freq
0                   Café  0.25
1    Indie Movie Theater  0.12
2  Australian Restaurant  0.12


----Ainslie----
               venue  freq
0                Pub  0.14
1  Fish & Chips Shop  0.14
2      Grocery Store  0.14


----Amaroo----
                venue  freq
0  Athletics & Sports   0.5
1            Pharmacy   0.5
2             Airport   0.0


----Aranda----
               venue  freq
0               Café   0.4
1  Recreation Center   0.2
2       Dance Studio   0.2


----Australian National University----
         venue  freq
0  Coffee Shop  0.20
1         Café  0.17
2   Food Truck  0.07


----Banks----
           venue  freq
0    Pizza Place   0.5
1  Grocery Store   0.5
2        Airport   0.0


----Barton----
         venue  freq
0         Café  0.25
1        Hotel  0.21
2  Coffee Shop  0.08


----Beard----
              venue  freq
0  Business Service  0.25
1              Pool  0.25
2             Motel  0.25


----Belconnen----
        

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [138]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cbr_grouped['Neighborhood']

for ind in np.arange(cbr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cbr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Acton,Café,Hotel Bar,Indie Movie Theater,Museum,Australian Restaurant
1,Ainslie,Pub,Grocery Store,Café,Australian Restaurant,Bakery
2,Amaroo,Athletics & Sports,Pharmacy,Yoga Studio,Flower Shop,Furniture / Home Store
3,Aranda,Café,Recreation Center,Dance Studio,Bar,Yoga Studio
4,Australian National University,Coffee Shop,Café,Food Truck,Beer Bar,Gaming Cafe


In [139]:
# cluster neighbourhoods

# set number of clusters
kclusters = 5

cbr_grouped_clustering = cbr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cbr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

array([3, 0, 0, 3, 3], dtype=int32)

In [0]:
# add clustering labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

In [141]:
neighborhoods_venues_sorted.tail()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
128,Watson,3,Shopping Plaza,Café,Filipino Restaurant,Fish & Chips Shop,Grocery Store
129,Weston,3,Café,Cricket Ground,Sandwich Place,Fast Food Restaurant,Yoga Studio
130,Woden,3,Martial Arts Dojo,Business Service,Café,Paper / Office Supplies Store,Furniture / Home Store
131,Wreck Bay,3,Café,Gym / Fitness Center,Gift Shop,History Museum,Fish Market
132,Wright,2,River,Yoga Studio,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop


In [142]:
# merge dataset and check output

cbr_merged = pd.merge(cbr, neighborhoods_venues_sorted, on='Neighborhood')
cbr_merged.tail()

,Neighborhood,Postcode,Country,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
131,Ginninderra Village,2913,Australia,Australian Capital Territory,-35.231177,149.081971,3,Café,Yoga Studio,Flower Shop,Fruit & Vegetable Store,Frozen Yogurt Shop
132,Palmerston,2913,Australia,Australian Capital Territory,-35.197250,149.117580,0,Baseball Field,Grocery Store,Bus Station,Ethiopian Restaurant,Food Court
133,Ngunnawal,2913,Australia,Australian Capital Territory,-35.173190,149.108020,3,Grocery Store,Café,Fast Food Restaurant,Yoga Studio,Food & Drink Shop
134,Nicholls,2913,Australia,Australian Capital Territory,-35.184180,149.099160,0,Grocery Store,Sporting Goods Shop,Resort,Soccer Field,Yoga Studio
135,Amaroo,2914,Australia,Australian Capital Territory,-35.169220,149.126370,0,Athletics & Sports,Pharmacy,Yoga Studio,Flower Shop,Furniture / Home Store


In [143]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cbr_merged['Latitude'], cbr_merged['Longitude'], cbr_merged['Neighborhood'], cbr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [144]:
# cluster 0
cbr_merged.loc[cbr_merged['Cluster Labels'] == 0, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,2614,149.048260,0,Motel,Bus Station,Vietnamese Restaurant,Steakhouse,Yoga Studio
5,2601,149.130930,0,Coffee Shop,Café,Beer Bar,Shopping Mall,Chocolate Shop
17,2602,149.146550,0,Pub,Grocery Store,Café,Australian Restaurant,Bakery
18,2602,149.115310,0,Sports Club,Bus Stop,Business Service,Yoga Studio,Dog Run
21,2602,149.126790,0,Pub,Gym / Fitness Center,Café,Chinese Restaurant,Tennis Court
24,2603,149.146100,0,Hotel,Café,Vietnamese Restaurant,French Restaurant,Australian Restaurant
27,2604,149.152400,0,Shopping Mall,Asian Restaurant,Italian Restaurant,Mediterranean Restaurant,Yoga Studio
28,2604,149.150364,0,Park,Burger Joint,Museum,Train Station,Go Kart Track
29,2605,149.108750,0,Grocery Store,Bakery,Malay Restaurant,Yoga Studio,Food & Drink Shop
30,2605,149.094940,0,Fast Food Restaurant,Bakery,Yoga Studio,Flower Shop,Furniture / Home Store


In [145]:
# cluster 1
cbr_merged.loc[cbr_merged['Cluster Labels'] == 1, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
55,2612,149.16005,1,Bus Stop,Yoga Studio,Flower Shop,Fruit & Vegetable Store,Frozen Yogurt Shop
66,2615,149.02221,1,Bus Stop,Yoga Studio,Flower Shop,Fruit & Vegetable Store,Frozen Yogurt Shop
87,2904,149.09226,1,Breakfast Spot,Bus Stop,Yoga Studio,Garden,Furniture / Home Store


In [146]:
# cluster 2
cbr_merged.loc[cbr_merged['Cluster Labels'] == 2, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,2611,149.04089,2,River,Yoga Studio,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop
52,2611,149.04240,2,River,Yoga Studio,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop


In [147]:
# cluster 3
cbr_merged.loc[cbr_merged['Cluster Labels'] == 3, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,2600,149.133540,3,Café,Hotel,Coffee Shop,History Museum,Cantonese Restaurant
1,221,149.133540,3,Café,Hotel,Coffee Shop,History Museum,Cantonese Restaurant
2,2600,149.126550,3,Café,Gym / Fitness Center,Gift Shop,History Museum,Fish Market
3,2601,149.126550,3,Café,Gym / Fitness Center,Gift Shop,History Museum,Fish Market
6,2540,149.144621,3,Pub,Café,Food Truck,Park,Trail
7,2540,149.108064,3,Café,Grocery Store,Sports Bar,Pizza Place,Gas Station
8,2540,149.126550,3,Café,Gym / Fitness Center,Gift Shop,History Museum,Fish Market
9,2600,149.126680,3,Café,Gym / Fitness Center,Gift Shop,History Museum,Fish Market
10,2600,149.126550,3,Café,Gym / Fitness Center,Gift Shop,History Museum,Fish Market
11,2600,149.149920,3,Plaza,Playground,Café,Coffee Shop,Yoga Studio


In [148]:
# cluster 4
cbr_merged.loc[cbr_merged['Cluster Labels'] == 4, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
70,2615,149.04430,4,Convenience Store,Yoga Studio,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop
110,2617,149.09891,4,Convenience Store,Yoga Studio,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop
124,2905,149.12418,4,Convenience Store,Yoga Studio,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop


In [149]:
# cluster 5
cbr_merged.loc[cbr_merged['Cluster Labels'] == 5, cbr_merged.columns[[1] + list(range(5, cbr_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
